<a href="https://colab.research.google.com/github/yuneess/vsf-assets/blob/main/CS6315_Final_Project_Schedule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

=== Progress Log ===

251107
*   Migrated back to Python
*   Completed mapping current process of scheduling
*   Ran a test with a few requirements
*   Started reviewing the sudoku sample

=== To Do ===
*   Finish adding explanation of all decision points
*   Bring in the input data - labor database and other estimating info
*   Figure out how to bring in the logic identified for each path into z3
*   Optimization
*   Presentation prep


#Verification of Construction Schedule Feasibility



Construction schedule planning operates under dual constraints: the scope of work (determined by design specifications) and the target completion date (set by client requirements).  This project develops a formal verification system to determine whether construction tasks can be completed within target milestones given real-world constraints.  

[![Overview](https://raw.githubusercontent.com/yuneess/vsf-assets/refs/heads/main/images/Overview.jpg)](https://raw.githubusercontent.com/yuneess/vsf-assets/refs/heads/main/images/Overview.jpg)



The system will use Z3 SMT solver to verify schedule feasibility by modeling:

* Initial State: Quantity-based duration estimates (QD) derived from material quantities, base productivity rates, and initial crew assignments
* Target State: Required completion duration (TD) specified by project milestones
* Decision Variables: Adjustable parameters including crew count and work hours
* Constraints: Physical workspace limitations, crew availability, and productivity factors

The verification process follows the decision logic in the flowchart, systematically exploring crew adjustment strategies when QD > TD:

* Extending work hours with existing crews
* Adding additional crews (subject to availability and workspace capacity)

Each path through the decision tree leads to either an accepting state (feasible schedule) or a rejection state (infeasible within constraints).  Beyond binary feasibility checking, the system will compute cost implications for each satisfiable solution and, when multiple solutions exist, identify optimal configurations based on prioritized objectives (minimize cost, minimize duration, etc.).

Implementation will use Python's Z3 library in Google Colab, enabling formal verification of construction schedules through constraint satisfaction and optimization.

## Decision Points in Current Workflow
The workflow diagram below represents the current construction scheduling process, developed through direct observations and discovery sessions with estimators and schedulers, and subsequently validated by these domain experts.

[![WorkFlow](https://raw.githubusercontent.com/yuneess/vsf-assets/refs/heads/main/images/WorkFlow.jpg)](
https://raw.githubusercontent.com/yuneess/vsf-assets/refs/heads/main/images/WorkFlow.jpg)

The workflow assumes that working more hours with the same assigned crew, if possible, is prefered over adding more crews to the job site.  Thus, it goes through "add more hours" route before it goes into checking "add more people" - black sticky note at the decision point C.

It is also assumed that this project starts at where data is transferred from estimators to schedulers - red arrow towards the top in the diagram.

##Formal Specification

[![DecisionPoints](https://raw.githubusercontent.com/yuneess/vsf-assets/refs/heads/main/images/DecisionPoints.jpg)](https://raw.githubusercontent.com/yuneess/vsf-assets/refs/heads/main/images/DecisionPoints.jpg)


**DECISION POINT A: DO WE HAVE ENOUGH CREW AVAILABLE?**  
If we don't have the number crew that was specified in the base crew count used in estimating, we will not be able to perform the task.  Thus, we want to check, at minimum, if we have the base crew that can work on this task.

To verify this we want to check:  
*(Labor Availability) >= (Base Labor Count)*

**DECISION POINT B: CAN THE WORK BE COMPLETED WITHIN TARGET DURATION?**  
This is the initial check.  Once the estimate duration (ED), which is calculated based on the material quantity and base metrics, is passed down, it needs to be compared with the target duration (TD).  If ED is less or equal to TD, it is good to go - we can move onto the next work area.  If ED is greater than TD, we need to verify if it is possible to complete the work within TD by working more hours and/or adding more people to the task.

To verify this, we want to check:  
*(Estimated Duration) <= (Target duration)*

**DECISION POINT C: CAN EXISTING CREW WORK MORE TIME?**  
Overtime rate for the same Labor exist?

**DECISION POINT D: CAN THE WORK BE COMPLETED WITH THE ADJUSTMENT?**  
(Adjusted Duration) <= (Target duration)

**DECISION POINT E: CAN THE WORK AREA ACCOMMODATE MORE PEOPLE?**  
(Current Labor Count) <= (Work Area Max Occupancy)

**DECISION POINT F: EXTRA CREWS AVAILABLE?**  
((Labor Availability) - (Current Labor Count)) > 1

**DECISION POINT G: CAN THE WORK BE COMPLETED WITH THE ADJUSTMENT?**  
(Adjusted Duration) <= (Target duration)

From the simplified diagram above, the diagram below shows the all paths to SAT or UNSAT.

<a href="https://raw.githubusercontent.com/yuneess/vsf-assets/refs/heads/main/images/AllPaths.jpg">
<img src="https://raw.githubusercontent.com/yuneess/vsf-assets/refs/heads/main/images/AllPaths.jpg" alt="All Paths" width="600">
</a>

**PATH 1:**  
¬A == UNSAT

**PATH 2:**  
A ∧ B == SAT

**PATH 3:**  
A ∧ ¬B ∧ C ∧ D == SAT

**PATH 4:**  
A ∧ ¬B ∧ ¬C ∧ ¬E == UNSAT

**PATH 5:**  
A ∧ ¬B ∧ C ∧ ¬D ∧ ¬E == UNSAT

**PATH 6:**  
A ∧ ¬B ∧ ¬C ∧ E ∧ ¬F == UNSAT

**PATH 7:**  
A ∧ ¬B ∧ C ∧ ¬D ∧ E ∧ ¬F == UNSAT

**PATH 8:**  
A ∧ ¬B ∧ ¬C ∧ E ∧ F ∧ ¬G == UNSAT

**PATH 9:**  
A ∧ ¬B ∧ C ∧ ¬D ∧ E ∧ F ∧ ¬G == UNSAT

**PATH 10:**  
A ∧ ¬B ∧ ¬C ∧ E ∧ F ∧ G == SAT

**PATH 11:**  
A ∧ ¬B ∧ C ∧ ¬D ∧ E ∧ F ∧ G == SAT  

  
**=WORK IN PROGRESS=**

##Formal Model

In [1]:
# Import z3
!pip install z3-solver
from z3 import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 66.7 MB/s eta 0:00:00


In [2]:
from typing import Dict
# Variables

# Material database and labor database to be brought in as dictionary
Labor_DB = Dict[str, float] # will probably need another data structure

Material_Cost = Real('Material_Cost')
Labor_Cost = Real('Labor_Cost')
Total_Direct_Cost = Real('Total_Direct_Cost')
Total_Indirect_Cost = Real('Total_Indirect_Cost')
Total_Cost = Real('Total_Cost')

Labor_Availability = Int('Labor_Availability')
Base_Labor_Count = Int('Base_Labor_Count')
Current_Labor_Count = Int('Current_Crew_Count')

# All durations are in hours
# All calculations are to be done in hours and roll up to 8-hour day when needed
Estimated_Duration = Int('Estimated_Duration')
Target_Duration = Int('Target_Duration')
Adjusted_Duration = Int('Adjusted_Duration')
Adjusted_Duration <= Estimated_Duration

Work_Area_Max_Occupancy = Int('Work_Area_Max_Occupancy')

##### WORK IN PROGRESS #####

In [11]:
# Invariants
Inv_01 = Labor_Availability >= 0
Inv_02 = Current_Labor_Count >= 0
Inv_03 = Base_Labor_Count >= 0

Inv_04 = Estimated_Duration >= 0
Inv_05 = Target_Duration >= 0
Inv_06 = Adjusted_Duration >= 0

Inv_07 = Material_Cost # do i need to note the value that shouldn't change?
Inv_08 = Material_Cost = Total_Cost - Total_Indirect_Cost - Labor_Cost # making sure material cost doesn't change

Prod_Deduction_Rate = Real('Prod_Deduction') # Assumption: if increased by more than x2, deduction rate applies to productivity

##### WORK IN PROGRESS #####

In [ ]:
# Add a block of code so that I can take the real data
# Building Core Data (BCD)
# Labor Database (LD)

In [12]:
# Requirements
# DECISION POINT A: DO WE HAVE ENOUGH CREW AVAILABLE?
dec_A = (Labor_Availability) >= (Base_Labor_Count)

# DECISION POINT B: CAN THE WORK BE COMPLETED WITHIN TARGET DURATION?
dec_B = (Estimated_Duration) <= (Target_Duration)

# DECISION POINT C: CAN EXISTING CREW WORK MORE TIME?**
# dec_C = Overtime rate for the same crew exist?
# need to make an interface-like structure where one object contains the following:
# Labor_Mix
# Labor_Productivity
# Labor_Reg_Rate
# Labor_Over_Rate


# DECISION POINT D: CAN THE WORK BE COMPLETED WITH THE ADJUSTMENT?
dec_D = (Adjusted_Duration) <= (Target_Duration)

# DECISION POINT E: CAN THE WORK AREA ACCOMMODATE MORE PEOPLE?
dec_E = (Current_Labor_Count) <= (Work_Area_Max_Occupancy)

# DECISION POINT F: EXTRA CREWS AVAILABLE?**
dec_F = ((Labor_Availability) - (Current_Labor_Count)) > 1

# DECISION POINT G: CAN THE WORK BE COMPLETED WITH THE ADJUSTMENT?
dec_G = (Adjusted_Duration) <= (Target_Duration)

##### WORK IN PROGRESS #####

In [21]:
# Verification test
s = Solver()

# Invariants
# Safety Req; "nothing bad ever happens"
s.add(Labor_Availability >= 0)
s.add(Current_Labor_Count >= 0)
s.add(Base_Labor_Count >= 0)

s.add(Estimated_Duration >= 0)
s.add(Target_Duration >= 0)
s.add(Adjusted_Duration >= 0)

s.add(Material_Cost == Total_Cost - Total_Indirect_Cost - Labor_Cost) # making sure material cost doesn't change

Prod_Deduction_Rate = Real('Prod_Deduction') # Assumption: if increased by more than x2, deduction rate applies to productivity

# Constraints
s.assert_and_track(Labor_Availability == 5, 'c1')
s.assert_and_track(Base_Labor_Count == 4, 'c2') # initial data transmitted from estimates

s.assert_and_track(Estimated_Duration == 11, 'c3') # initial data transmitted from estimates
s.assert_and_track(Target_Duration == 12, 'c4')

s.assert_and_track(dec_A, 'dA')
s.assert_and_track(dec_B, 'dB')

if s.check() == sat:
    print("SAT")
    m = s.model()
    print(m)
else:
    print("UNSAT")
    print(f"unsat_core: {s.unsat_core()}")

SAT
[Current_Crew_Count = 0,
 Target_Duration = 12,
 Adjusted_Duration = 0,
 Estimated_Duration = 11,
 dA = True,
 c2 = True,
 c3 = True,
 c4 = True,
 Labor_Availability = 5,
 Base_Labor_Count = 4,
 c1 = True,
 dB = True]


In [3]:
# sample
# from: https://stackoverflow.com/questions/23451388/z3-sudoku-solver

# 9x9 matrix of integer variables
X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ]
      for i in range(9) ]
print(X)

# each cell contains a value in {1, ..., 9}
cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9)
             for i in range(9) for j in range(9) ]

# each row contains a digit at most once
rows_c   = [ Distinct(X[i]) for i in range(9) ]
# distinct basically encodes: X[0][0] != X[1][0] != X[2][0] ...

# each column contains a digit at most once
cols_c   = [ Distinct([ X[i][j] for i in range(9) ])
             for j in range(9) ]
# distinct basically encodes: X[0][0] != X[0][1] != X[0][2] ...


# each 3x3 square contains a digit at most once
sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j]
                        for i in range(3) for j in range(3) ])
             for i0 in range(3) for j0 in range(3) ]

sudoku_c = cells_c + rows_c + cols_c + sq_c

# sudoku instance, we use '0' for empty cells
"""
instance = ((5,3,0,0,7,0,0,0,0),
            (6,0,0,1,9,5,0,0,0),
            (0,9,8,0,0,0,0,6,0),
            (8,0,0,0,6,0,0,0,3),
            (4,0,0,8,0,3,0,0,1),
            (7,0,0,0,2,0,0,0,6),
            (0,6,0,0,0,0,2,8,0),
            (0,0,0,4,1,9,0,0,5),
            (0,0,0,0,8,0,0,7,9))
"""

# unconstrained: any sudoku solution
instance = ((0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0))


# unsatisfiable: 2 ones in 1st column
"""
instance = ((1,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (0,0,0,0,0,0,0,0,0),
            (1,0,0,0,0,0,0,0,0))
"""


instance_c = [ If(instance[i][j] == 0,
                  True,
                  X[i][j] == instance[i][j])
               for i in range(9) for j in range(9) ]

s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
          for i in range(9) ]
    print_matrix(r)

    # could put this next bit into a loop: keep preventing previous solutions
    # from being used, by constraining variables to not equal previous
    # solutions (models) found
    # prevent next model from being equal to previous: generate other
    # solutions
    c_different = [ And([ X[i][j] != m.evaluate(X[i][j]) for j in range(9) ])
          for i in range(9) ]

    print(c_different)
    s.add(c_different)

    s.check()

    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
          for i in range(9) ]

    print_matrix(r)
else:
    print("failed to solve")

[[x_1_1, x_1_2, x_1_3, x_1_4, x_1_5, x_1_6, x_1_7, x_1_8, x_1_9], [x_2_1, x_2_2, x_2_3, x_2_4, x_2_5, x_2_6, x_2_7, x_2_8, x_2_9], [x_3_1, x_3_2, x_3_3, x_3_4, x_3_5, x_3_6, x_3_7, x_3_8, x_3_9], [x_4_1, x_4_2, x_4_3, x_4_4, x_4_5, x_4_6, x_4_7, x_4_8, x_4_9], [x_5_1, x_5_2, x_5_3, x_5_4, x_5_5, x_5_6, x_5_7, x_5_8, x_5_9], [x_6_1, x_6_2, x_6_3, x_6_4, x_6_5, x_6_6, x_6_7, x_6_8, x_6_9], [x_7_1, x_7_2, x_7_3, x_7_4, x_7_5, x_7_6, x_7_7, x_7_8, x_7_9], [x_8_1, x_8_2, x_8_3, x_8_4, x_8_5, x_8_6, x_8_7, x_8_8, x_8_9], [x_9_1, x_9_2, x_9_3, x_9_4, x_9_5, x_9_6, x_9_7, x_9_8, x_9_9]]
[[8, 6, 4, 7, 2, 9, 5, 3, 1],
 [9, 1, 2, 4, 5, 3, 7, 6, 8],
 [3, 7, 5, 6, 1, 8, 2, 4, 9],
 [6, 4, 9, 8, 7, 5, 3, 1, 2],
 [7, 2, 1, 9, 3, 6, 8, 5, 4],
 [5, 3, 8, 2, 4, 1, 6, 9, 7],
 [4, 8, 6, 5, 9, 7, 1, 2, 3],
 [1, 9, 7, 3, 6, 2, 4, 8, 5],
 [2, 5, 3, 1, 8, 4, 9, 7, 6]]
[And(8 != x_1_1,
    6 != x_1_2,
    4 != x_1_3,
    7 != x_1_4,
    2 != x_1_5,
    9 != x_1_6,
    5 != x_1_7,
    3 != x_1_8,
    1 != x_1_9)

Z3Exception: model is not available